In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
#from keras import ops
from keras.layers import TextVectorization

In [3]:
with open('barnette_graph_data_alpha.npy') as f:
    barnette_graphs = [line.rstrip('\n') for line in f]

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 723: character maps to <undefined>

In [ ]:
barnette_graphs[0]

In [ ]:
with open('hamiltonian_cycle_data.txt') as f:
    lines_hamiltonian_data = [line.rstrip('\n') for line in f]

In [ ]:
hamiltonian_cycles=[]
i=0
for line in lines_hamiltonian_data:
    hamiltonian_cycles.append("s " +lines_hamiltonian_data[i].strip(')').strip('(').split(',')[0][1:-1]+ " e")
    i=i+1

In [ ]:
hamiltonian_cycles[0]

In [ ]:
len(barnette_graphs)

In [ ]:
len(hamiltonian_cycles)

In [9]:
max_barnette_len=0
for j in range(len(barnette_graphs)):
    if len(barnette_graphs[j])>max_barnette_len:
        max_barnette_len=len(barnette_graphs[j])

In [10]:
max_barnette_len

684

In [11]:
barnette_vocab=[
'0',
'1',
'2',
'3',
'4',
'5',
'6',
'7',
'8',
'9',
"'",
'[',
']',
'{',
'}',
':',
',',
' ',
'p',
''
]

In [12]:
max_hamiltonian_len=0
for j in range(len(hamiltonian_cycles)):
    if len(hamiltonian_cycles[j])>max_hamiltonian_len:
        max_hamiltonian_len=len(hamiltonian_cycles[j])

In [13]:
hamiltonian_vocab=['s',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'e',
 '.',
 ' ',
 'p',
  '']

In [14]:
max_hamiltonian_len

37

In [15]:
barnette_vocab_dict={}
for i in range(len(barnette_vocab)):
    barnette_vocab_dict[barnette_vocab[i]]=i

In [16]:
barnette_vocab_rev_dict={}
for i in range(len(barnette_vocab)):
    barnette_vocab_rev_dict[i]=barnette_vocab[i]

In [17]:
hamiltonian_vocab_dict={}
for i in range(len(hamiltonian_vocab)):
    hamiltonian_vocab_dict[hamiltonian_vocab[i]]=i

In [18]:
hamiltonian_vocab_rev_dict={}
for i in range(len(hamiltonian_vocab)):
    hamiltonian_vocab_rev_dict[i]=hamiltonian_vocab[i]

In [19]:
barnette_vocab_dict

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 "'": 10,
 '[': 11,
 ']': 12,
 '{': 13,
 '}': 14,
 ':': 15,
 ',': 16,
 ' ': 17,
 'p': 18,
 '': 19}

In [20]:
hamiltonian_vocab_dict

{'s': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'e': 11,
 '.': 12,
 ' ': 13,
 'p': 14,
 '': 15}

In [21]:
def text_to_numeric(data,vocab_dict,max_len):
    data_numeric=[]
    for i in range(len(data)):
        numeric=[]

        for char in data[i]:
            numeric.append(vocab_dict[char])

        count=len(numeric)

        while count<=max_len:
            numeric.append(vocab_dict['p'])
            count=count+1
        data_numeric.append(np.array(numeric))

    return np.array(data_numeric)

In [22]:
hamiltonian_cycles_numeric=text_to_numeric(hamiltonian_cycles,hamiltonian_vocab_dict, 2*max_hamiltonian_len)

In [23]:
hamiltonian_cycles_numeric.shape

(50000, 75)

In [24]:
barnette_graphs_numeric=text_to_numeric(barnette_graphs,barnette_vocab_dict, 800)

In [25]:
barnette_graphs_numeric.shape

(50000, 801)

In [26]:
num_val_samples = int(0.15 * len(hamiltonian_cycles))
num_train_samples = len(hamiltonian_cycles) - 2 * num_val_samples

barnette_graphs_trn = barnette_graphs_numeric[:num_train_samples]
hamiltonian_cycles_trn = hamiltonian_cycles_numeric[:num_train_samples]

barnette_graphs_val = barnette_graphs_numeric[num_train_samples : num_train_samples + num_val_samples]
hamiltonian_cycles_val = hamiltonian_cycles_numeric[num_train_samples : num_train_samples + num_val_samples]

barnette_graphs_tst = barnette_graphs_numeric[num_train_samples + num_val_samples :]
hamiltonian_cycles_tst = hamiltonian_cycles_numeric[num_train_samples + num_val_samples :]


print(f"{len(hamiltonian_cycles)} total samples")
print(f"{len(hamiltonian_cycles_trn)} training samples")
print(f"{len(hamiltonian_cycles_val)} validation pairs")
print(f"{len(hamiltonian_cycles_tst)} test pairs")

50000 total samples
35000 training samples
7500 validation pairs
7500 test pairs


In [27]:

#import keras.ops as ops


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:,tf.newaxis, tf.newaxis, :], dtype='int32')
            #ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        #ops.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        #ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return tf.math.not_equal(inputs,0)
        #ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
            #ops.cast(mask[:, None, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
            #ops.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        #ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        #ops.arange(sequence_length)[:, None]
        j = tf.range(sequence_length)
        #ops.arange(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        #ops.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        #ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        #mult = ops.concatenate(
        #    [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
        #    axis=0,
        #)
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.convert_to_tensor([1, 1])],
            axis=0,
        )
        return tf.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [28]:
embed_dim = 256
latent_dim = 2048
num_heads = 1

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(2*max_barnette_len, len(barnette_vocab), embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(2*max_hamiltonian_len, len(hamiltonian_vocab), embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(len(hamiltonian_vocab), activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [29]:
epochs = 30  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit([barnette_graphs_trn, hamiltonian_cycles_trn[:, :-1]], hamiltonian_cycles_trn[:, 1:], epochs=epochs, validation_data=([barnette_graphs_val, hamiltonian_cycles_val[:, :-1]], hamiltonian_cycles_val[:, 1:]))

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃     Param # ┃ Connected to                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs (InputLayer)   │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ positional_embedding          │ (None, None, 256)         │     355,328 │ encoder_inputs[0][0]           │
│ (PositionalEmbedding)         │                           │             │                                │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ decoder_inputs (InputLayer)   │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ transformer_encoder           │ (None, None, 256)         │   1,315,072 │ positional_embedding[0][0]     │
│ (TransformerEncoder)          │                           │             │                                │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ functional_5 (Functional)     │ (None, None, 16)          │   1,605,904 │ decoder_inputs[0][0],          │
│                               │                           │             │ transformer_encoder[0][0]      │
└───────────────────────────────┴───────────────────────────┴─────────────┴────────────────────────────────┘

 Total params: 3,276,304 (12.50 MB)

 Trainable params: 3,276,304 (12.50 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2998s 3s/step - accuracy: 0.9403 - loss: 0.2152 - val_accuracy: 0.9606 - val_loss: 0.1158
Epoch 2/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2819s 3s/step - accuracy: 0.9630 - loss: 0.1071 - val_accuracy: 0.9641 - val_loss: 0.1026
Epoch 3/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2825s 3s/step - accuracy: 0.9644 - loss: 0.1021 - val_accuracy: 0.9663 - val_loss: 0.0989
Epoch 4/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2819s 3s/step - accuracy: 0.9669 - loss: 0.0952 - val_accuracy: 0.9685 - val_loss: 0.0907
Epoch 5/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2832s 3s/step - accuracy: 0.9696 - loss: 0.0881 - val_accuracy: 0.9673 - val_loss: 0.0939
Epoch 6/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2808s 3s/step - accuracy: 0.9698 - loss: 0.0874 - val_accuracy: 0.9699 - val_loss: 0.0863
Epoch 7/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2804s 3s/step - accuracy: 0.9708 - loss: 0.0838 - val_accuracy: 0.9705 - val_loss: 0.0847
Epoch 8/30
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 2805s 3s/step - accuracy: 0.9716 -

KeyboardInterrupt: 

In [30]:
barnette_vocab=barnette_vocab_dict
barnette_index_lookup = dict(zip(range(len(barnette_vocab)), barnette_vocab))
max_decoded_sentence_length = 2*max_hamiltonian_len

In [31]:
def decode_sequence(input_sentence):

    tokenized_input_sentence = np.array([input_sentence])
    tokenized_target_sentence=np.zeros(len(hamiltonian_cycles[0]))+hamiltonian_vocab_dict['']
    tokenized_target_sentence[0]=hamiltonian_vocab_dict['s']
    tokenized_target_sentence=np.array([tokenized_target_sentence])

    decoded_sentence = "s"

    for i in range(max_decoded_sentence_length):
        #print(i)
        tokenized_target_sentence = tokenized_target_sentence[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # ops.argmax(predictions[0, i, :]) is not a concrete value for jax here
        # sampled_token_index = ops.convert_to_numpy(
        #    ops.argmax(predictions[0, i, :])
        #).item(0)

        sampled_token_index =np.argmax(predictions[0, i, :])
        sampled_token = barnette_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        print(decoded_sentence)

        if sampled_token == "e":
            break
    return decoded_sentence

In [36]:
for i in range(1):
    input_barnette_graph = barnette_graphs_trn[3]
    predicted_hamiltonian_cycle = decode_sequence(input_barnette_graph)

s {
s { 2
s { 2 ]
s { 2 ] ]
s { 2 ] ] ]
s { 2 ] ] ] {
s { 2 ] ] ] { {
s { 2 ] ] ] { { {
s { 2 ] ] ] { { { {
s { 2 ] ] ] { { { { ]
s { 2 ] ] ] { { { { ] {


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 11 of dimension 1 out of bounds. [Op:StridedSlice] name: strided_slice/

In [33]:
barnette_graphs_val[i]

array([13, 10,  1, 10, 15, 17, 11,  1, 16, 17,  2, 16, 17,  1,  2, 16, 17,
        1,  1, 16, 17,  1,  6, 16, 17,  1,  5, 16, 17,  4, 16, 17,  3, 12,
       16, 17, 10,  2, 10, 15, 17, 11,  5, 16, 17,  6, 16, 17,  2, 16, 17,
        1, 12, 16, 17, 10,  3, 10, 15, 17, 11,  9, 16, 17,  1,  0, 16, 17,
        1,  1, 16, 17,  1,  2, 12, 16, 17, 10,  4, 10, 15, 17, 11,  3, 16,
       17,  4, 16, 17,  8, 16, 17,  2,  1, 16, 17,  2,  2, 16, 17,  7, 12,
       16, 17, 10,  5, 10, 15, 17, 11,  5, 16, 17,  1, 16, 17,  3, 16, 17,
        7, 12, 16, 17, 10,  6, 10, 15, 17, 11,  3,  3, 16, 17,  3,  4, 16,
       17,  3,  5, 16, 17,  3,  6, 12, 16, 17, 10,  7, 10, 15, 17, 11,  1,
        3, 16, 17,  1,  4, 16, 17,  1,  5, 16, 17,  1,  6, 12, 16, 17, 10,
        8, 10, 15, 17, 11,  1,  2, 16, 17,  2, 16, 17,  6, 16, 17,  1,  9,
       16, 17,  2,  0, 16, 17,  9, 12, 16, 17, 10,  9, 10, 15, 17, 11,  1,
        4, 16, 17,  1,  7, 16, 17,  1,  8, 16, 17,  8, 16, 17,  4, 16, 17,
        1,  5, 12, 16, 17